In [300]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sample_submission.csv
/kaggle/input/test.csv
/kaggle/input/train.csv


In [301]:
import pandas as pd
import numpy as np

In [302]:
df = pd.read_csv('/kaggle/input/train.csv')

In [303]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [304]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [305]:
df = df.drop(['id','keyword','location'],axis=1)

In [306]:
df

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [307]:
import string
def rem_punct(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation , '')
    return text
df['text'] = df['text'].apply(rem_punct)
        

In [308]:
import nltk
from nltk.corpus import stopwords

In [309]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [310]:
from nltk.stem.porter import PorterStemmer

In [311]:
stemmer = PorterStemmer()

In [312]:
stop = set(stopwords.words('english'))
def stem_text(text):
    f_text = []
    for i in text.split():    # split the sentenses into List With commas
        if i.strip().lower not in stop:        #strip means you wanna strip/delete that word from list of split or delete whitespaces if no agument is passed
            word = stemmer.stem(i.strip())
            f_text.append(word)
    return " ".join(f_text)

df['text'] = df['text'].apply(stem_text)
    
            
    

In [313]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#from wordcloud import WordCloud , STOPWORDS
#plt.figure(figsize = (20,20))
#wc = WordCloud(max_words=4000 , width=1200 , height=6500, stopwords = STOPWORDS ).generate(" ".join(df.text))
#plt.imshow(wc , interpolation = 'bilinear')

In [314]:
df["text"] = df['text'].str.replace('[^\w\s]','')
df['text'] = df['text'].str.replace('\d+', '')
df['text'] = df["text"].str.lower()

In [315]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [316]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.target)

In [317]:
tv = TfidfVectorizer()
X = tv.fit_transform(x_train)
Y = tv.transform(x_test)
print(len(tv.get_feature_names()))

15220


In [318]:
print(X.shape)

(5709, 15220)


In [325]:
from sklearn.decomposition import PCA 
pca = PCA(n_components = 2) 
pca.fit(X.toarray()) 
x_pca = pca.transform(X.toarray()) 
x_pca.shape   


(5709, 2)

In [327]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = x_pca.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.fit(x_pca,y_train , epochs = 10)

Epoch 1/10
5709/5709 [==============================] - 1s 204us/step - loss: 0.6325 - accuracy: 0.6586
Epoch 2/10
5709/5709 [==============================] - 1s 142us/step - loss: 0.6011 - accuracy: 0.6721
Epoch 3/10
5709/5709 [==============================] - 1s 143us/step - loss: 0.6006 - accuracy: 0.6723
Epoch 4/10
5709/5709 [==============================] - 1s 139us/step - loss: 0.5988 - accuracy: 0.6781
Epoch 5/10
5709/5709 [==============================] - 1s 158us/step - loss: 0.5946 - accuracy: 0.6754
Epoch 6/10
5709/5709 [==============================] - 1s 146us/step - loss: 0.5943 - accuracy: 0.6786
Epoch 7/10
5709/5709 [==============================] - 1s 147us/step - loss: 0.5952 - accuracy: 0.6768
Epoch 8/10
5709/5709 [==============================] - 1s 138us/step - loss: 0.5925 - accuracy: 0.6858
Epoch 9/10
5709/5709 [==============================] - 1s 139us/step - loss: 0.5957 - accuracy: 0.6791
Epoch 10/10
5709/5709 [==============================] - 1s 138u

In [328]:
pca = PCA(n_components = 2) 
pca.fit(Y.toarray()) 
y_pca = pca.transform(Y.toarray()) 
y_pca.shape  

(1904, 2)

In [329]:
pred = model.predict(y_pca)

for i in range(len(pred)):
    if(pred[i] > 0.5):
        pred[i] = 1
    else:
        pred[i] = 0
        

In [330]:
pred

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

In [331]:
from sklearn.metrics import accuracy_score 
accuracy_score(pred,y_test)

0.6901260504201681

In [332]:
#pred.tolist()

In [333]:
from sklearn.tree import DecisionTreeClassifier

In [334]:
dtc  = DecisionTreeClassifier()

In [335]:
dtc.fit(x_pca,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [336]:
pred_dtc = dtc.predict(y_pca)
pred_dtc

array([0, 0, 0, ..., 1, 0, 0])

In [337]:
from sklearn.metrics import accuracy_score 
import sklearn.metrics as metrics

In [338]:
print("Accuracy:",metrics.accuracy_score(y_test, pred_dtc))

Accuracy: 0.6066176470588235


In [339]:

from sklearn.linear_model import LogisticRegression

In [340]:
lr = LogisticRegression()
lr.fit(x_pca,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [341]:
pred_lr = lr.predict(y_pca)
pred_lr

array([0, 0, 0, ..., 1, 0, 0])

In [342]:
print("Accuracy:",metrics.accuracy_score(y_test, pred_lr))

Accuracy: 0.6796218487394958


In [343]:
from sklearn.ensemble import RandomForestClassifier

In [344]:
rfc = RandomForestClassifier()

In [345]:
rfc.fit(x_pca,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [346]:
pred_rfc = rfc.predict(y_pca)
pred_rfc

array([1, 0, 0, ..., 1, 0, 0])

In [347]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, pred_rfc))

Accuracy: 0.6538865546218487


In [348]:
df_test = pd.read_csv('/kaggle/input/test.csv')

In [349]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [350]:
df_test = df_test.drop(['id','keyword','location'],axis=1)

In [351]:
import string
def rem_punct(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation , '')
    return text
df_test['text'] = df_test['text'].apply(rem_punct)
        

In [352]:
import nltk
from nltk.corpus import stopwords

In [353]:
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [354]:
stop = set(stopwords.words('english'))
def stem_text(text):
    f_text = []
    for i in text.split():    # split the sentenses into List With commas
        if i.strip().lower not in stop:        #strip means you wanna strip/delete that word from list of split or delete whitespaces if no agument is passed
            word = stemmer.stem(i.strip())
            f_text.append(word)
    return " ".join(f_text)

df_test['text'] = df_test['text'].apply(stem_text)

In [355]:
df_test.head()

,text
0,just happen a terribl car crash
1,heard about earthquak is differ citi stay safe...
2,there is a forest fire at spot pond gees are f...
3,apocalyps light spokan wildfir
4,typhoon soudelor kill 28 in china and taiwan


In [356]:
df_test["text"] = df_test['text'].str.replace('[^\w\s]','')
df_test['text'] = df_test['text'].str.replace('\d+', '')
df_test['text'] = df_test["text"].str.lower()

In [357]:
df_test

,text
0,just happen a terribl car crash
1,heard about earthquak is differ citi stay safe...
2,there is a forest fire at spot pond gees are f...
3,apocalyps light spokan wildfir
4,typhoon soudelor kill in china and taiwan
...,...
3258,earthquak safeti lo angel ûò safeti fasten xrwn
3259,storm in ri wors than last hurrican my cityamp...
3260,green line derail in chicago httptcoutbxlcbiuy
3261,meg issu hazard weather outlook hwo httptcoxrb...


In [358]:
tv_test = TfidfVectorizer()
X_test = tv_test.fit_transform(df_test.text)


In [362]:
X_test.shape
pca = PCA(n_components = 2) 
pca.fit(X_test.toarray()) 
xtest_pca = pca.transform(X_test.toarray()) 
xtest_pca.shape  

(3263, 2)

In [370]:
res = lr.predict(xtest_pca)
res

array([0, 0, 1, ..., 1, 0, 0])

In [371]:
xtest_pca.shape

(3263, 2)

In [378]:
res1 = res.tolist()
res1

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,


In [379]:
Submission = pd.read_csv('/kaggle/input/sample_submission.csv')

In [380]:
Submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [381]:
Submission = Submission.drop('target',axis=1)

In [382]:
Submission

,id
0,0
1,2
2,3
3,9
4,11
...,...
3258,10861
3259,10865
3260,10868
3261,10874


In [384]:
Submission['target'] = res1

In [387]:
Submission.to_csv('Tweet1.csv',index=False)